# Analyse sequentielle sur des données de métro

Objectif: prédire la ville à partir des observations

Le jeu de données du métro de Hangzhou décrit le flux entrant et sortant de  80 stations de la ville agrégée par quart d'heure entre $5h30$ et $23h30$ chaque jour. Deux tenseurs sont dans l'archive, un d'apprentissage et l'autre de test. Ils sont  de taille $D\times T \times S \times 2$ avec $D$ le nombre de jour, $T=73$ les tranches successives de quart d'heure entre $5h30$ et $23h30$, $S=80$ le nombre  de stations et les flux entrant et sortant pour la dernière dimension.

On va travailler sur un **sous-échantillon pour simplifier les choses**: ce sous-échantillon est paramétré dans les boites ci-dessous.

L'architecture du réseau est fournie (dans le fichier annexe utils.py), l'enjeu sera de compléter la fonction standard d'apprentissage pour vous adapter à une architecture définie.

In [ ]:
from utils import RNN, device,SampleMetroDataset
import torch
from torch.utils.data import DataLoader

In [ ]:
import numpy as np
import torch.nn as nn
import torch.optim
from torch.utils.data import DataLoader
import logging
import tensorboard
from torch.utils.tensorboard import SummaryWriter
import time
from itertools import chain
logging.basicConfig(level=logging.INFO)


In [ ]:
from pathlib import Path
from IPython.display import display, HTML
from torch.utils.tensorboard import SummaryWriter

# Chemin vers TensorBoard
TB_PATH = "/tmp/logs"

# TENSORBOARD (en dehors du notebook => Navigateur)
# usage externe de tensorboard: (1) lancer la commande dans une console; (2) copier-coller l'URL dans un navigateur
display(HTML("<h2>Informations</h2><div>Pour visualiser les logs, tapez la commande : </div>"))
print(f"tensorboard --logdir {Path(TB_PATH).absolute()}")
print("Une fois la commande lancer dans la console, copier-coller l'URL dans votre navigateur")

## A. Chargement d'un sous-échantillon des données

In [ ]:
# Nombre de stations utilisé 
CLASSES = 10
#Longueur des séquences 
LENGTH = 20
# Dimension de l'entrée (1 (in) ou 2 (in/out))
DIM_INPUT = 2
#Taille du batch
BATCH_SIZE = 32

PATH = "./data/"

matrix_train, matrix_test = torch.load(open(PATH+"hzdataset.pch","rb"))
ds_train = SampleMetroDataset(matrix_train[:, :, :CLASSES, :DIM_INPUT], length=LENGTH)
ds_test = SampleMetroDataset(matrix_test[:, :, :CLASSES, :DIM_INPUT], length = LENGTH, stations_max = ds_train.stations_max)
data_train = DataLoader(ds_train,batch_size=BATCH_SIZE,shuffle=True)
data_test = DataLoader(ds_test, batch_size=BATCH_SIZE,shuffle=False)


In [ ]:
# analyse des données
# quelques propositions... Mais à vous de jouer

print(len(ds_train))
print(ds_train[0])
print(ds_train[0][0].size())

cl = [ds_train[i][1] for i in range(len(ds_train))]

print(np.unique(cl))
print(ds_train[1][0].sum(0))

In [ ]:

# paramértage par défaut
dim_input=2
epochs=100
batch_size=32
length=20
latent=10
classes=10


## Définition du modèle

**ATTENTION** Le modèle récurrent est défini dans le fichier joint... Mais vous noterez ici que le décodeur est déclaré à part... Ce qui aura une incidence certaine sur l'inférence du modèle et donc sur la boucle d'apprentissage.

In [ ]:
rnn = RNN(dim_input,latent) # cf code dans utils.py
decoder = nn.Linear(latent,classes)
loss = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(chain(rnn.parameters(),decoder.parameters()),lr=0.0001)
writer = SummaryWriter(TB_PATH+"/predictStations-"+time.asctime())


## B. Apprentissage

L'enjeu est de faire rentrer ce réseau de neurones dans la boucle d'apprentissage classique développée dans les séances précédante.

1. Récupérer une boucle d'apprentissage standard dans un TP précédent
2. Jouer avec le modèle rnn pour comprendre ses entrées et ses sorties
    - vérifier les dimensions de sortie du réseau
    - choisir où brancher le décodeur
3. Itérer jusqu'à réussir à lancer l'apprentissage

Vérifier les performances directement sur tensorboard au bout de l'apprentissage

In [ ]:
# boucle standard d'apprentissage
# 


def train(rnn, decoder, epochs):
    rnn = rnn.to(device)
    decoder = decoder.to(device)
##<CORRECTION>
    for epoch in range(epochs):
        
        logging.info("Iteration %d", epoch)
        suml = 0
        err = 0
        for x,y in data_train: 
            optim.zero_grad()
            x = x.to(device)
            y = y.to(device)
            #if dim_input==1:
            #    x = x[:,:,0]
            h = rnn.forward(x.transpose(0,1))[-1]
            yhat = decoder.forward(h)
            l = loss(yhat,y)
            suml += l/len(data_train)
            err += (yhat.max(1)[1]!=y).float().mean().item()*1./len(data_train)
            l.backward()
            optim.step()
        writer.add_scalar('loss/train',suml,epoch)
        writer.add_scalar('error/train',err,epoch)
        logging.info("loss train : %f -- %f",suml,err)
        with torch.no_grad():
            l = 0
            err=0
            for x,y in data_test:
                x = x.to(device)
                y = y.to(device)
                #if dim_input==1:
                #    x = x[:,:,0]
                yhat = decoder.forward(rnn.forward(x.transpose(0,1))[-1])
                l += loss(yhat,y)/len(data_test)
                err+=(yhat.max(1)[1]!=y).double().mean().item()*1./len(data_test)
            writer.add_scalar('loss/test',l,epoch)
            writer.add_scalar('error/test',err,epoch)
            logging.info("loss test : %f --%f",l,err)
    return rnn
##</CORRECTION>

In [ ]:
train(rnn, decoder, epochs)

### Evaluation des performances

Comparer les performances en apprentissage et en test en utilisant le taux de bonne classification à l'issue de l'apprentissage.

Le système présente-il des symptomes de sur-apprentissage? Dans l'affirmative, quelle procédure aurait-on pu mettre en place?



In [ ]:
#<CORRECTION>
# inutile dans la correction les erreurs sont calculées pendant l'apprentissage
err = 0
for x,y in data_train: 
    x = x.to(device)
    y = y.to(device)
    h = rnn.forward(x.transpose(0,1))[-1]
    yhat = decoder.forward(h)
    err += (yhat.max(1)[1]!=y).float().mean().item()*1./len(data_train)
print("Erreur d'apprentissage: ", err)

err=0
for x,y in data_test:
    x = x.to(device)
    y = y.to(device)
    
    yhat = decoder.forward(rnn.forward(x.transpose(0,1))[-1])
    err+=(yhat.max(1)[1]!=y).double().mean().item()*1./len(data_test)

print("Erreur de test: ", err)

# il y a beaucoup de sur-apprentissage. A minima, il faut faire de l'early stopping

#</CORRECTION>

## C. Prediction d'affluence

L'objectif de cette partie est de faire de la prédiction de séries temporelles : à partir d'une séquence de flux de longueur $t$ pour l'ensemble des stations du jeu de données, on veut prédire le flux à $t+1$, $t+2$, $\ldots$. Vous entraînerez un RNN commun à toutes les stations qui prend une série dans $\mathbb{R}^{n\times 2}$ et  prédit une série dans $\mathbb{R}^{n\times 2}$.

Que doit-on changer au modèle précédent ? Quel coût est dans ce cas plus adapté que la cross-entropie ? 

 Faire les expériences en faisant varier l'horizon de prédiction (à $t+2$, etc.) et la longueur des séquences en entrée. Vous pouvez comme précédemment  considérer d'abord que le flux entrant, puis le flux entrant et sortant.


Dans ce contexte de réseau \textit{many-to-many}, la supervision peut se faire à chaque étape de la séquence sans attendre la fin de la séquence. La rétro-propagation n'est faîte qu'une fois que toute la séquence a été vue, mais à un instant $t$, le gradient prend en compte l'erreur à ce moment (en fonction de la supervision du décodage) mais également l'erreur des pas de temps d'après qui est cumulée. 

In [ ]:
# paramértage par défaut
dim_input=2
epochs=100
batch_size=32
length=20
latent=10
classes=10

stations = 20
length=20
length_fc=10

In [ ]:
from utils import RNN, device,  ForecastMetroDataset

matrix_train, matrix_test = torch.load(open(PATH+"hzdataset.pch","rb"))
ds_train = ForecastMetroDataset(matrix_train[:,:,:stations,:dim_input],length=length)
ds_test  = ForecastMetroDataset(matrix_test[:,:,:stations,:dim_input],length=length,stations_max=stations)

data_train = DataLoader(ds_train,batch_size=batch_size,shuffle=True)
data_test  = DataLoader(ds_test,batch_size=batch_size,shuffle=False)


### Analyser en détail les dimensions des données d'apprentissage pour comprendre comment apprendre sur ces données

Votre code doit vous mener à quelque chose comme la figure ci-dessous:

![ts-metro](data/xy-teacherforcing.png)


In [ ]:
import matplotlib.pyplot as plt

# Extraire un echantillon du dataset, mesurer les tailles de x et y... Eventuellement faire un plot

# <CORRECTION>
x,y = ds_train[0]

print("x:",x.size())
print("y:",y.size())

# premieres series du premier batch
plt.figure(figsize=(12,4))
for i in range (4):
    plt.subplot(1,4,i+1)
    plt.plot(x[:,i,0], label="x")
    plt.plot(y[:,i,0], label="y")
    plt.legend()
plt.savefig("data/xy-teacherforcing.png")
# </CORRECTION>

In [ ]:
# on fournit le code du prédicteur qui permet d'exploiter la structure du réseau de neurones
# et de ré-injecter la sortie prédite pour prédire les valeurs suivantes

# evidemment, le code est dépendant d'un décodeur qu'il faut déinir

def forecast(rnn,decoder,x,h=None,length=10):
    with torch.no_grad():
        if h is None:
            h = rnn.hzero(x.size(1)).to(x.device)
        h = rnn.forward(x,h)[-1]
        x = decoder.forward(h)
        yhat = [x]
        for i in range(length-1):
            x = decoder.forward(rnn.one_step(x,h))
            yhat.append(x)
    return torch.stack(yhat)

## Définition du modèle

On vous laisse définir le modèle... Etant donné que le décodeur est à part, prendre le temps de réfléchir à la nature du réseau récurrent. Il faut aussi déterminer la loss la mieux adaptée à ce nouveau problème.

In [ ]:

# rnn = 
# decoder =
# loss = 

##<CORRECTION>
rnn = RNN(dim_input*stations,latent)
decoder = nn.Linear(latent,dim_input*stations)
loss = torch.nn.MSELoss()
##</CORRECTION>

optim = torch.optim.Adam(chain(rnn.parameters(),decoder.parameters()),lr=0.0001)
writer = SummaryWriter(PATH+"/Forecast-"+time.asctime())


In [ ]:
def train(rnn, decoder, epochs):
##<CORRECTION>
    rnn = rnn.to(device)
    decoder = decoder.to(device)
    for epoch in range(epochs):
        logging.info("Iteration %d", epoch)
        suml = 0
        for x,y in data_train:
            # x :  batch x length x stations x dim_input
            l=0
            optim.zero_grad()
            x = x.view(x.size(0),x.size(1),-1).to(device)
            h = rnn.forward(x.transpose(0,1))
            yhat = decoder.forward(h.view(-1,latent)).view(x.size(1), x.size(0), x.size(2))
            l+= loss(yhat,y.view(y.size(0),y.size(1),-1).transpose(0,1).to(device))
            suml+=l/len(data_train)
            l.backward()
            optim.step()
        writer.add_scalar('loss/train',suml,epoch)
        logging.info("loss train : %f",suml)

        with torch.no_grad():
            l = 0
            lf = 0
            for x,y in data_test:
                x = x.view(x.size(0),x.size(1),-1).to(device)
                h = rnn.forward(x.transpose(0,1))
                yhat = decoder.forward(h.view(-1,latent)).view(x.size(1),x.size(0),x.size(2))
                l += loss(yhat,y.view(y.size(0),y.size(1),-1).transpose(0,1).to(device))/len(data_test)
                yhat = forecast(rnn,decoder,x.transpose(0,1)[:-length_fc],length=length_fc)
                lf += loss(yhat,y.view(y.size(0),y.size(1),-1).transpose(0,1).to(device)[-length_fc:])/len(data_test)
            writer.add_scalar('loss/test',l,epoch)
            logging.info("loss test : %f",l)
            writer.add_scalar('loss/test_fc',lf,epoch)
            logging.info("loss test forecast : %f",lf)
##</CORRECTION>
    return rnn

In [ ]:
train(rnn, decoder, epochs)

## Métrique et exploitation des résultats

Quelle métrique vous semble adaptée à ce problème? 

Ca vaut le coup de tester la MAPE (mean absolute percentage error) et de vérifier les problèmes autour des heures de faible affluence... Pour ensuite regarder ce qui existe comme alternatives (SMAPE, ...)

[Mértique alternative](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error)<BR>
[lien vers des explications sur les métriques de régression](https://kobia.fr/regression-metrics-quelle-metrique-choisir/)

## Calculer les métriques en fonction de l'horizon de prédiction

Utiliser la méthode `forecast` pour prédire à des horizons supérieurs à 1... Et analyser les métriques en fonction de l'horizon.

# Construction du sujet à partir de la correction

In [2]:
### <CORRECTION> ###
import re
# transformation de cet énoncé en version étudiante

fname = "4_1_RNN-metro-corr.ipynb" # ce fichier
fout  = fname.replace("-corr","")

# print("Fichier de sortie: ", fout )

f = open(fname, "r")
txt = f.read()
 
f.close()

f2 = open(fout, "w")
f2.write(re.sub("<CORRECTION>.*?(</CORRECTION>)"," TODO ",\
    txt, flags=re.DOTALL))
f2.close()

### </CORRECTION> ###